In [1]:
!pip install datasets
!pip install transformers
!pip install tqdm 

distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None


In [3]:
import random
import numpy as np
from pprint import pprint

from datasets import load_dataset, load_metric
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
dataset = load_dataset('squad_kor_v1')

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [5]:
corpus = list(set([example['context'] for example in dataset['train']]))
corpus.extend(list(set([example['context'] for example in dataset['validation']])))

tokenizer_func = lambda x: x.split(' ')
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))
sp_matrix = vectorizer.fit_transform(corpus)

In [7]:
def get_relevant_doc(vectorizer, query, k=1):
    "참고 : vocab에 없는 이상한 단어로 query하는 경우 assertion발생 예)뙣뙇?"
    
    query_vec = vectorizer.transform([query])
    assert np.sum(query_vec) !=0
    result = query_vec * sp_matrix.T
    sorted_result = np.argsort(-result.data)
    doc_scores = result.data[sorted_result]
    doc_ids = result.indices[sorted_result]
    return doc_scores[:k],doc_ids[:k]

In [8]:
query = "미국의 대통령은 누구인가?"

_,doc_id = get_relevant_doc(vectorizer, query,k=1)
print("{} {} {}".format('*'*20, 'result','*'*20))
print("[Search query]\n",query,'\n')
print(f"[Relevant Doc Id(Top 1)]:{doc_id.item()}")
print(corpus[doc_id.item()])

******************** result ********************
[Search query]
 미국의 대통령은 누구인가? 

[Relevant Doc Id(Top 1)]:3642
영국과 미국의 관계는 대략 400년 정도 소급된다. 1607년 영국은 "제임스타운"이라고 명명된 북미 대륙 최초의 상주 식민지를 세우기 시작하였고, 오늘날 영국과 미국은 가까운 군사적 동맹체다. 양국은 문화적으로 유사할 뿐만 아니라 군사적 연구와 정보 기구를 공유한다. 영국은 미국으로부터 토마호크 미사일이나 트라이던트 미사일과 같은 무기를 구입했고 미국은 영국으로부터 해리어(Harrier)와 같은 장비를 구매했다. 또한 영국에 대규모 군대를 주둔하고 있다. 최근 영국의 총리와 미국의 대통령은 매우 친밀한 모습을 보여주었다. 가령 토니 블레어와 빌 클린턴 및 이후 조지 W. 부시 대통령 간의 관계, 1980년대 마거릿 대처와 로널드 레이건 등의 관계가 그러하다. 현재 영국의 정책은 혈맹 미국과의 관계는 영국의 "가장 중요한 2자간 관계"임을 표명한다.


In [13]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [14]:
model_name = "sangrimlee/bert-base-multilingual-cased-korquad"

mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
)
mrc_model = mrc_model.eval()

In [19]:
def get_answer_from_context(context,question,model,tokenizer):
    encoded_dict = tokenizer.encode_plus(
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    non_padded_ids = encoded_dict["input_ids"][:encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)
    inputs = {
        'input_ids':torch.tensor([encoded_dict['input_ids']],dtype=torch.long),
        'attention_mask':torch.tensor([encoded_dict['attention_mask']],dtype=torch.long),
        'token_type_ids':torch.tensor([encoded_dict['token_type_ids']],dtype=torch.long),
    }
    outputs=model(**inputs)
    start, end = torch.max(outputs.start_logits,axis=1).indices.item(),torch.max(outputs.end_logits,axis=1).indices.item()
    answer=tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [20]:
context = corpus[doc_id.item()]

In [21]:
context

'영국과 미국의 관계는 대략 400년 정도 소급된다. 1607년 영국은 "제임스타운"이라고 명명된 북미 대륙 최초의 상주 식민지를 세우기 시작하였고, 오늘날 영국과 미국은 가까운 군사적 동맹체다. 양국은 문화적으로 유사할 뿐만 아니라 군사적 연구와 정보 기구를 공유한다. 영국은 미국으로부터 토마호크 미사일이나 트라이던트 미사일과 같은 무기를 구입했고 미국은 영국으로부터 해리어(Harrier)와 같은 장비를 구매했다. 또한 영국에 대규모 군대를 주둔하고 있다. 최근 영국의 총리와 미국의 대통령은 매우 친밀한 모습을 보여주었다. 가령 토니 블레어와 빌 클린턴 및 이후 조지 W. 부시 대통령 간의 관계, 1980년대 마거릿 대처와 로널드 레이건 등의 관계가 그러하다. 현재 영국의 정책은 혈맹 미국과의 관계는 영국의 "가장 중요한 2자간 관계"임을 표명한다.'

In [22]:
answer = get_answer_from_context(context,query,mrc_model,tokenizer)
print(answer)

조지 W. 부시


In [24]:
def open_domain_qa(query,corpus,vectorizer,model,tokenizer,k=1):
    
    _,doc_id = get_relevant_doc(vectorizer, query,k=1)
    context = corpus[doc_id.item()]
    
    
    print("{} {} {}".format('*'*20, 'result','*'*20))
    print("[Search query]\n",query,'\n')
    print(f"[Relevant Doc Id(Top 1)]:{doc_id.item()}")
    pprint(corpus[doc_id.item()],compact=True)
    print(f"[Answer prediction from the model]:{answer}")

In [25]:
query = input("Enter any question:")
open_domain_qa(query,corpus,vectorizer,mrc_model,tokenizer,k=1)

Enter any question: 미국의 대통령은 누구인가? 


******************** result ********************
[Search query]
 미국의 대통령은 누구인가?  

[Relevant Doc Id(Top 1)]:3642
('영국과 미국의 관계는 대략 400년 정도 소급된다. 1607년 영국은 "제임스타운"이라고 명명된 북미 대륙 최초의 상주 식민지를 세우기 '
 '시작하였고, 오늘날 영국과 미국은 가까운 군사적 동맹체다. 양국은 문화적으로 유사할 뿐만 아니라 군사적 연구와 정보 기구를 공유한다. '
 '영국은 미국으로부터 토마호크 미사일이나 트라이던트 미사일과 같은 무기를 구입했고 미국은 영국으로부터 해리어(Harrier)와 같은 장비를 '
 '구매했다. 또한 영국에 대규모 군대를 주둔하고 있다. 최근 영국의 총리와 미국의 대통령은 매우 친밀한 모습을 보여주었다. 가령 토니 '
 '블레어와 빌 클린턴 및 이후 조지 W. 부시 대통령 간의 관계, 1980년대 마거릿 대처와 로널드 레이건 등의 관계가 그러하다. 현재 '
 '영국의 정책은 혈맹 미국과의 관계는 영국의 "가장 중요한 2자간 관계"임을 표명한다.')
[Answer prediction from the model]:조지 W. 부시


In [26]:
query = None
while query != "그만":
    query = input("Enter any question:")
    open_domain_qa(query,corpus,vectorizer,mrc_model,tokenizer,k=1)

Enter any question: 미국의 대통령은 누구인가?


******************** result ********************
[Search query]
 미국의 대통령은 누구인가? 

[Relevant Doc Id(Top 1)]:3642
('영국과 미국의 관계는 대략 400년 정도 소급된다. 1607년 영국은 "제임스타운"이라고 명명된 북미 대륙 최초의 상주 식민지를 세우기 '
 '시작하였고, 오늘날 영국과 미국은 가까운 군사적 동맹체다. 양국은 문화적으로 유사할 뿐만 아니라 군사적 연구와 정보 기구를 공유한다. '
 '영국은 미국으로부터 토마호크 미사일이나 트라이던트 미사일과 같은 무기를 구입했고 미국은 영국으로부터 해리어(Harrier)와 같은 장비를 '
 '구매했다. 또한 영국에 대규모 군대를 주둔하고 있다. 최근 영국의 총리와 미국의 대통령은 매우 친밀한 모습을 보여주었다. 가령 토니 '
 '블레어와 빌 클린턴 및 이후 조지 W. 부시 대통령 간의 관계, 1980년대 마거릿 대처와 로널드 레이건 등의 관계가 그러하다. 현재 '
 '영국의 정책은 혈맹 미국과의 관계는 영국의 "가장 중요한 2자간 관계"임을 표명한다.')
[Answer prediction from the model]:조지 W. 부시


Enter any question: 그만


******************** result ********************
[Search query]
 그만 

[Relevant Doc Id(Top 1)]:1048
('폭스는 세계적인 섹스 심볼로 불리고 있는데, 《엠파이어》와의 인터뷰에서 "올바른 선택을 하고 열심히 공부해 스스로를 개발하고 싶다"는 '
 '메건 폭스는 "남성잡지의 표지 모델도 그만 하고 싶다. 이제는 작품성 있는 영화로 인정받겠다"라고 말했고, 이어 "시간을 섹시스타로 살 '
 '수는 없다"고 말하며 섹스 심볼이라는 이미지에 대해 반감을 드러냈다. 또한 폭스는 섹시한 이미지로 남자 팬들을 자극할 수는 있지만, '
 '여성들에게는 좋지 않은 인상으로 "여성들은 내가 멍청하거나 아니면 그저 외모지상주의자라는 선입견을 갖고 있다. 남성들에게 어필하는 방법은 '
 '잘 알고 있지만 여성들이 무조건 나를 싫어하는 이유는 잘 모르겠다"라고 속내를 밝혔다.')
[Answer prediction from the model]:조지 W. 부시
